![LipPike](https://upload.wikimedia.org/wikipedia/commons/3/32/Lip_Pike_Baseball.jpg "Lip Pike had Style")

### Back in the Good Ole Days of Base Ball...

In [1]:
cd ..

/Users/johnphillips/Desktop/DSI-Class-Stuff/BaseBall


In [2]:
%matplotlib inline

In [3]:
%run __init__.py

In [4]:
# Start with reading the data sets from the pickles

teams = pd.read_pickle('data/teams.p') # Full Data
teams_num = pd.read_pickle('data/teams_num.p') # Numeric Features
new_reduced = pd.read_pickle('data/new_reduced.p') # Reduced from PCA
salaries = pd.read_pickle('data/salaries.p') # Team Salaries 1985 - 2016
rockies = pd.read_pickle('data/rockies.p') # 2017 Rockies Team Stats

### Can I find which team was the best pre 1903?

Plan of Action:
* Average proper features
* Scale
* PCA only with same features
* Fit_transform new_teams
* Transform old_teams
* CV STEP: 20% of Years --> Test
   * 80% of Years --> Train
   * Test Years Individually...
   
* Apply Model to old_teams, year by year.
* Successful if picks winner that is also LCS Winner...
* Who would win WS?


In [5]:
# Look at all the columns, and then decide which ones I want to keep:
teams.columns

Index([u'yearID', u'teamID', u'franchID', u'G', u'Ghome', u'W', u'L', u'LgWin',
       u'WSWin', u'R', u'AB', u'H', u'2B', u'3B', u'HR', u'BB', u'SO', u'SB',
       u'CS', u'HBP', u'SF', u'RA', u'ER', u'ERA', u'CG', u'SHO', u'SV',
       u'IPouts', u'HA', u'HRA', u'BBA', u'SOA', u'E', u'DP', u'FP', u'name',
       u'attendance', u'BPF', u'PPF', u'teamIDBR', u'teamIDlahman45',
       u'teamIDretro', u'BA', u'OBP', u'SLG', u'OPS', u'WinP'],
      dtype='object')

In [6]:
# Numeric Columns that I want to keep for plots:
#teams = teams.select_dtypes(exclude=['object', 'category'])     

### Average some columns...
* The older teams didn't play as many games as modern teams.
* I want to look at each stat as a performance per game...

In [7]:
# Columns to Avg
columns_to_avg = ['W', 'L', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'E', 'DP', 'FP', 'attendance' ]

# teams.drop(['Ghome', 'franchID', 'CG', 'DivWin', 'WCWin'], axis=1, inplace=True)

In [8]:
for col in columns_to_avg:
    teams[col] = (teams[col] / teams['G']).replace()

In [9]:
teams.head(9)

,yearID,teamID,franchID,G,Ghome,W,L,LgWin,WSWin,R,...,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,BA,OBP,SLG,OPS,WinP
0,1871,BS1,BNA,31,0.0,0.645161,0.322581,0,0,12.935484,...,103,98,BOS,BS1,BS1,0.310496,0.339385,0.422012,0.761397,0.666667
1,1871,CH1,CNA,28,0.0,0.678571,0.321429,0,0,10.785714,...,104,102,CHI,CH1,CH1,0.270067,0.304936,0.373746,0.678682,0.678571
2,1871,CL1,CFC,29,0.0,0.344828,0.655172,0,0,8.586207,...,96,100,CLE,CL1,CL1,0.276560,0.292079,0.391231,0.683310,0.344828
3,1871,FW1,KEK,19,0.0,0.368421,0.631579,0,0,7.210526,...,101,107,KEK,FW1,FW1,0.238606,0.270860,0.293566,0.564426,0.368421
4,1871,NY2,NNA,33,0.0,0.484848,0.515152,0,0,9.151515,...,90,88,NYU,NY2,NY2,0.287037,0.303410,0.349715,0.653125,0.484848
5,1871,PH1,PNA,28,0.0,0.750000,0.250000,1,0,13.428571,...,102,98,ATH,PH1,PH1,0.320062,0.343632,0.434817,0.778449,0.750000
6,1871,RC1,ROK,25,0.0,0.160000,0.840000,0,0,9.240000,...,97,99,ROK,RC1,RC1,0.264479,0.290503,0.363900,0.654402,0.160000
7,1871,TRO,TRO,29,0.0,0.448276,0.517241,0,0,12.103448,...,101,100,TRO,TRO,TRO,0.307692,0.333847,0.417468,0.751315,0.464286
8,1871,WS3,OLY,32,0.0,0.468750,0.468750,0,0,9.687500,...,94,98,OLY,WS3,WS3,0.277162,0.301927,0.368810,0.670737,0.500000


### Separate into Modern Teams of World Series Era, and Old Teams for pre-WS ERA:

In [10]:
# Start with finding old teams:

old_teams = teams[(teams['yearID'] < 1903)]
new_teams = teams[(teams['yearID'] > 1902)]

# Show old_teams along with year, name and some key features.
old_teams.loc[:, ('yearID', 'name', 'WinP', 'R', 'RA', 'ERA', 'SHO', 'SLG', 'attendance', 'LgWin', 'WSWin' )].head(9)

,yearID,name,WinP,R,RA,ERA,SHO,SLG,attendance,LgWin,WSWin
0,1871,Boston Red Stockings,0.666667,12.935484,9.774194,3.55,0.032258,0.422012,0.0,0,0
1,1871,Chicago White Stockings,0.678571,10.785714,8.607143,2.76,0.000000,0.373746,0.0,0,0
2,1871,Cleveland Forest Citys,0.344828,8.586207,11.758621,4.11,0.000000,0.391231,0.0,0,0
3,1871,Fort Wayne Kekiongas,0.368421,7.210526,12.789474,5.17,0.052632,0.293566,0.0,0,0
4,1871,New York Mutuals,0.484848,9.151515,9.484848,3.72,0.030303,0.349715,0.0,0,0
5,1871,Philadelphia Athletics,0.750000,13.428571,9.500000,4.95,0.000000,0.434817,0.0,1,0
6,1871,Rockford Forest Citys,0.160000,9.240000,11.480000,4.30,0.040000,0.363900,0.0,0,0
7,1871,Troy Haymakers,0.464286,12.103448,12.482759,5.51,0.000000,0.417468,0.0,0,0
8,1871,Washington Olympics,0.500000,9.687500,9.468750,4.37,0.000000,0.368810,0.0,0,0


In [11]:
# Some basic stats on the old teams:

old_teams[['R', 'RA', 'ERA', 'SHO', 'PPF', 'OPS', 'attendance']].describe()

,R,RA,ERA,SHO,PPF,OPS,attendance
count,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000,415.000000
mean,5.820068,6.043688,3.547349,0.049053,100.346988,0.652594,545.880756
std,1.584602,2.114944,1.019881,0.036782,6.029323,0.079398,822.821084
min,1.944444,2.833333,1.220000,0.000000,60.000000,0.336493,0.000000
25%,4.831560,4.848850,2.815000,0.022222,97.000000,0.603263,0.000000
50%,5.553957,5.630435,3.440000,0.044118,100.000000,0.654051,0.000000
75%,6.481199,6.505459,4.175000,0.067925,104.000000,0.708371,1108.285574
max,13.428571,25.333333,8.000000,0.250000,141.000000,0.889770,3571.210526


In [12]:
# By comparison... How do the teams compare since the World Series started?

new_teams[['R', 'RA', 'ERA', 'SHO', 'PPF', 'OPS', 'attendance']].describe()

,R,RA,ERA,SHO,PPF,OPS,attendance
count,2420.000000,2420.000000,2420.000000,2420.000000,2420.000000,2420.000000,2420.000000
mean,4.393718,4.394111,3.865442,0.065985,100.202066,0.712585,8955.427333
std,0.643418,0.656107,0.696704,0.031333,4.599288,0.053116,5813.529492
min,2.409091,2.458065,1.730000,0.000000,88.000000,0.529958,0.000000
25%,3.962963,3.953115,3.420000,0.043210,97.000000,0.679162,4015.649887
50%,4.376543,4.351852,3.860000,0.061728,100.000000,0.714938,7811.769148
75%,4.786060,4.801579,4.320000,0.082064,103.000000,0.746710,13087.270100
max,6.896104,7.685897,6.710000,0.207792,129.000000,0.870036,28047.102564


   We can see the attendance has gone up dramatically.   
   Scoring and OPS have changed, but the old teams are still competitive.

In [13]:
# Compare Z-score for winning percentage and year...

def zscore(series):
    return (series - series.mean())/series.std()

def z_score_with_year_name(old_teams):
    df = pd.DataFrame(
    {'Z-WinP': zscore(old_teams['WinP']),
    'Year': old_teams['yearID'],
    'Name': old_teams['name'],
    'WinP': old_teams['WinP'], 
    'Z-Runs': zscore(old_teams['R']),
    'Z-OPS': zscore(old_teams['OPS']),
    'Z-ERA': zscore(old_teams['ERA'])
    })
    return df

In [14]:
zscore(old_teams['WinP']).head(10)

0    1.174887
1    1.253430
2   -0.948480
3   -0.792820
4   -0.024678
5    1.724687
6   -2.167899
7   -0.160343
8    0.075286
9    1.052709
Name: WinP, dtype: float64

In [15]:
old_teams['name'].head()

0       Boston Red Stockings
1    Chicago White Stockings
2     Cleveland Forest Citys
3       Fort Wayne Kekiongas
4           New York Mutuals
Name: name, dtype: object

In [16]:
old_teams.groupby('yearID').apply(z_score_with_year_name).head(30)

,Name,WinP,Year,Z-ERA,Z-OPS,Z-Runs,Z-WinP
0,Boston Red Stockings,0.666667,1871,-0.836335,1.093901,1.228448,0.939951
1,Chicago White Stockings,0.678571,1871,-1.752567,-0.146088,0.207945,1.003595
2,Cleveland Forest Citys,0.344828,1871,-0.186855,-0.076708,-0.836170,-0.780634
3,Fort Wayne Kekiongas,0.368421,1871,1.042519,-1.858917,-1.489210,-0.654500
4,New York Mutuals,0.484848,1871,-0.639171,-0.529218,-0.567816,-0.032067
5,Philadelphia Athletics,0.750000,1871,0.787366,1.349524,1.462519,1.385460
6,Rockford Forest Citys,0.160000,1871,0.033505,-0.510068,-0.525812,-1.768741
7,Troy Haymakers,0.464286,1871,1.436847,0.942763,0.833478,-0.141998
8,Washington Olympics,0.500000,1871,0.114690,-0.265191,-0.313382,0.048934
9,Baltimore Canaries,0.648148,1872,-0.793542,0.565189,0.897491,0.878318


In [17]:
old_teams.groupby('yearID').apply(z_score_with_year_name).tail(20)

,Name,WinP,Year,Z-ERA,Z-OPS,Z-Runs,Z-WinP
395,Philadelphia Phillies,0.592857,1901,-1.167156,-0.172771,-0.297580,0.957876
396,Pittsburgh Pirates,0.647482,1901,-1.711986,0.868476,0.743255,1.521299
397,St. Louis Cardinals,0.542857,1901,0.354609,0.642338,0.789949,0.442157
398,Washington Senators,0.458647,1901,1.124885,0.093989,-0.067401,-0.426424
399,Baltimore Orioles,0.362319,1902,1.844741,1.114444,0.831670,-1.343667
400,Boston Americans,0.562044,1902,-0.254378,0.741714,0.493408,0.610126
401,Brooklyn Superbas,0.543478,1902,-0.783164,-0.686115,-0.565221,0.428511
402,Boston Beaneaters,0.532847,1902,-0.911354,-0.848352,-0.528478,0.324508
403,Chicago White Sox,0.552239,1902,0.370551,0.071677,0.597380,0.514210
404,Chicago Orphans,0.496350,1902,-1.552307,-1.087363,-0.879753,-0.032514


In [18]:
standardized = old_teams.groupby(['yearID'])['WinP', 'R'].transform(zscore)

In [19]:
standardized.head(10)

,WinP,R
0,0.939951,1.228448
1,1.003595,0.207945
2,-0.780634,-0.836170
3,-0.654500,-1.489210
4,-0.032067,-0.567816
5,1.385460,1.462519
6,-1.768741,-0.525812
7,-0.141998,0.833478
8,0.048934,-0.313382
9,0.878318,0.897491


In [20]:
# Construct a Boolean Series to identify Good outliers: outliers
outliers = (standardized['WinP'] > 1) | (standardized['R'] > 1 ) 
# Filter old_teams by the outliers: old_outliers
old_outliers = old_teams.loc[outliers]

# Print gm_outliers
print(old_outliers)

     yearID teamID franchID    G  Ghome         W         L  LgWin  WSWin  \
0      1871    BS1      BNA   31    0.0  0.645161  0.322581      0      0   
1      1871    CH1      CNA   28    0.0  0.678571  0.321429      0      0   
5      1871    PH1      PNA   28    0.0  0.750000  0.250000      1      0   
12     1872    BS1      BNA   48    0.0  0.812500  0.166667      1      0   
16     1872    PH1      PNA   47    0.0  0.638298  0.297872      0      0   
17     1872    TRO      TRO   25    0.0  0.600000  0.400000      0      0   
20     1873    BL1      BLC   57    0.0  0.596491  0.385965      0      0   
23     1873    BS1      BNA   60    0.0  0.716667  0.266667      1      0   
31     1874    BS1      BNA   71    0.0  0.732394  0.253521      1      0   
38     1875    BS1      BNA   82    0.0  0.865854  0.097561      1      0   
44     1875    PH1      PNA   77    0.0  0.688312  0.259740      0      0   
51     1876    CHN      CHC   66    0.0  0.787879  0.212121      1      0   

In [21]:
old_outliers.shape

(90, 47)

In [22]:
old_outliers['LgWin'].sum(), old_teams['LgWin'].sum()

(46, 46)

### Split Data before Scaling:
* I don't want to scale Year, nor the target.

In [23]:
X_old_teams = old_teams[['W', 'L', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'E', 'DP', 'FP', 'attendance' ]
    ]

# Remember, no y value for Old Teams since no WS Winners pre 1903.

In [24]:
X_new_teams = new_teams[['W', 'L', 'R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'E', 'DP', 'FP', 'attendance' ]
    ]

y_new_teams = new_teams['WSWin']

### Scale our data

In [25]:
### Scale BOTH: old_teams and new_teams:

scale_old = StandardScaler().fit_transform(X_old_teams)
scale_old = pd.DataFrame(scale_old, columns=X_old_teams.columns)

scale_new = StandardScaler().fit_transform(X_new_teams)
scale_new = pd.DataFrame(scale_new, columns=X_new_teams.columns)

In [26]:
scale_new.tail(7)

,W,L,R,AB,H,2B,3B,HR,BB,SO,...,SHO,SV,IPouts,HA,HRA,BBA,E,DP,FP,attendance
2413,0.406082,-0.355563,0.539454,0.573434,0.040470,0.044557,-1.369749,1.856884,-0.186556,1.633246,...,-0.529979,1.430123,0.600164,-0.278226,1.816658,-0.749212,-1.120977,0.436304,-0.334069,0.867833
2414,0.482163,-0.431579,0.030877,0.421021,-0.042240,0.686742,0.372991,0.210985,0.587690,1.031052,...,0.061167,1.015194,0.815986,-0.951191,0.763971,-1.001489,-1.409347,-0.471292,-0.292519,2.033223
2415,0.406082,-0.355563,0.645008,0.277075,-0.244420,1.107483,-0.663233,1.892280,0.048064,1.733057,...,-0.135882,0.669419,0.039027,-0.083421,0.783111,-0.569014,-0.815643,0.890102,-0.361769,2.117371
2416,-0.963388,1.012721,-0.381743,-0.290240,-0.998001,0.863896,-0.663233,1.733000,-0.855222,2.278692,...,-0.529979,0.946039,-1.385396,-0.411048,1.759239,-0.376803,-1.036162,-0.760073,-0.347919,-0.174827
2417,1.090817,-1.039706,0.510667,0.082325,0.040470,0.177423,-1.087143,1.715302,-1.007725,1.407007,...,-0.924076,1.914208,-0.306287,-0.003727,1.586981,0.139765,-0.985273,1.756444,-0.347919,1.337751
2418,0.634327,-0.583611,0.453092,-0.307175,-0.768250,0.598165,-1.322648,1.821489,1.291549,1.879447,...,-0.135882,1.015194,0.751239,-0.898062,1.242465,-0.737199,-1.137940,-0.141257,-0.320219,2.061731
2419,1.090817,-1.039706,0.491475,-0.214033,-0.354700,0.421010,-0.804536,1.502928,0.165374,1.513473,...,0.258216,1.222658,0.772821,-1.500188,0.706552,-0.653106,-1.392384,-0.223766,-0.292519,1.095117


### PCA 

In [27]:
# fit_transform New
# fit Old

pca = PCA(n_components=5)
new_reduced_pca = pca.fit_transform(scale_new)
old_reduced_pca = pca.fit(scale_old)

In [28]:
print("New Reduced PCA shape: {}".format(str(new_reduced_pca.shape)))
#print("OLD Reduced PCA shape: {}".format(str(old_reduced_pca.shape)))

New Reduced PCA shape: (2420, 5)


In [29]:
old_reduced_pca

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [30]:
old_reduced_components = old_reduced_pca.components_
#old_reduced_components

### Cross Validation Step:

In [31]:
# for New data: scale_new or new_teams
# 20% of years to Test
# 80% of years to Train
# Test sets(years) indiviually:

# Train-test split our data...

# Selecting 20 years for Testing:
#test_years = ['1911', '1922', '1933', '1944', '1955', '1966', '1977', '1988', '1999', '2010', '1905', '1915', '1925', '1935', '1945', '1959', '1963', '1974', '1983', '1995']

#for year in test_years:
    #test_set = new_teams[(new_teams['yearID'] == year)]

# Standard split:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)



### Apply Model to Old Years, year by year
* Successful if it picks a NLCS winner... as WS Winner!
* Must apply to the 32 seasons before 1903.

### Can we predict which team would win the World Series before 1903?

In [32]:
len(old_teams_num['yearID'].unique()) 
# First remember in our old_teams data:
# We have only 32 Seasons, so we can only predict one winner per season...

NameError: name 'old_teams_num' is not defined

In [33]:
# Start with train_test_split with old_teams
old_teams_num = old_teams.select_dtypes(exclude=['object', 'category'])

# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X = old_teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 
y = old_teams_num['WSWin']